### *0. Imports*

In [2]:
import os
import shutil
import yaml
import numpy as np
from pathlib import Path
import torch
import torch.nn.functional as F
from pathlib import Path
import re


### *1. Data Cleaning* 

In [ ]:
"""Removing redundant data from GOOSE dataset download keeping only RGB images"""
#source_dir = Path(r"C:\Users\maxlars\UGV research pipeline\Goose_dataset\images\val")
#target_dir = Path(r"C:\Users\maxlars\UGV research pipeline\Goose_dataset\processed_rgb")

#target_dir.mkdir(parents=True, exist_ok=True)

#print("Starting trimming process...")
#
#for root, dirs, files in os.walk(source_dir):
#    for file in files:
#        if "vis" in file.lower() and file.endswith((".png", ".jpg")):
#            rel_path = os.path.relpath(root, source_dir)
#            dest_folder = target_dir / rel_path
#            dest_folder.mkdir(parents=True, exist_ok=True)
#            
#            shutil.copy2(os.path.join(root, file), dest_folder / file)
#
#print(f"Done! Cleaned RGB images are in: {target_dir}")

'Removing redundant data from GOOSE dataset download'

### *2. Projection*

In [4]:
import yaml
import torch
import torch.nn.functional as F
from pathlib import Path
import re

class GooseQueryProjector:
    def __init__(self, yaml_path, sequence_start_time_ns=0):
        self.yaml_path = Path(yaml_path)
        # Anchor for 4D T-dimension
        self.start_time_ns = sequence_start_time_ns 
        self.load_calibration()

    def load_calibration(self):
        """Loads camera parameters from the windshield_vis.yaml."""
        with open(self.yaml_path, 'r') as f:
            data = yaml.safe_load(f)
        
        # 1. Extracted Intrinsics
        self.img_width = data['image_width']
        self.img_height = data['image_height']
        self.K = torch.tensor(data['camera_matrix']['data']).view(3, 3).float()
        
        # 2. Distortion (Plumb Bob Model)
        self.D = torch.tensor(data['distortion_coefficients']['data']).float()
        
        # 3. Projection Matrix
        self.P = torch.tensor(data['projection_matrix']['data']).view(3, 4).float()

    def get_relative_timestamp(self, filename):
        """
        Parses nanoseconds from filename and calculates relative T for 4D Queries.
        Example: '..._1658494234334310308_windshield_vis.png'
        """
        match = re.search(r'(\d{19})', filename)
        if match:
            file_time_ns = int(match.group(1))
            # Convert to seconds relative to 'time_machine' start
            return (file_time_ns - self.start_time_ns) / 1e9
        return 0.0

    def undistort_image(self, image_tensor):
        """Pre-processes RGB images to align straight lines in off-road terrain."""
        h, w = image_tensor.shape[-2:]
        grid_y, grid_x = torch.meshgrid(
            torch.linspace(-1, 1, h), 
            torch.linspace(-1, 1, w), 
            indexing='ij'
        )
        
        u = (grid_x + 1) * (w - 1) / 2
        v = (grid_y + 1) * (h - 1) / 2
        
        x = (u - self.K[0, 2]) / self.K[0, 0]
        y = (v - self.K[1, 2]) / self.K[1, 1]
        
        r2 = x**2 + y**2
        radial = (1 + self.D[0]*r2 + self.D[1]*r2**2 + self.D[4]*r2**3)
        x_dist = x * radial + (2*self.D[2]*x*y + self.D[3]*(r2 + 2*x**2))
        y_dist = y * radial + (self.D[2]*(r2 + 2*y**2) + 2*self.D[3]*x*y)
        
        u_dist = self.K[0, 0] * x_dist + self.K[0, 2]
        v_dist = self.K[1, 1] * y_dist + self.K[1, 2]
        
        grid = torch.stack([2 * u_dist / (w - 1) - 1, 2 * v_dist / (h - 1) - 1], dim=-1).unsqueeze(0)
        return F.grid_sample(image_tensor, grid, align_corners=True)

    def project_4d_query(self, points_3d, ego_pose, relative_t):
        """Projects world-space voxels into 2D based on current UGV pose and time."""
        points_homo = torch.cat([points_3d, torch.ones(len(points_3d), 1)], dim=-1)
        # Transform P_world to P_camera
        points_cam = (ego_pose @ points_homo.T).T
        
        z = points_cam[:, 2:3].clamp(min=1e-3)
        pixels_2d = (self.K @ points_cam[:, :3].T).T / z
        
        return pixels_2d[:, :2], relative_t

### *3. Dataset*

In [8]:
from torch.utils.data import Dataset
from PIL import Image

class GooseTripletDataset(Dataset):
    def __init__(self, image_root, projector, transform=None):
        self.image_root = Path(image_root)
        self.projector = projector
        self.transform = transform
        self.samples = self._build_triplets()

    def _build_triplets(self):
        triplets = []
        # Iteratively traverse sorted subfolders
        for seq_folder in sorted(self.image_root.iterdir()):
            if not seq_folder.is_dir(): continue
            images = sorted([f for f in seq_folder.glob("*rgb.png")])
            
            # Create (prev, current, next) triplets
            for i in range(1, len(images) - 1):
                triplets.append({
                    'prev': images[i-1],
                    'curr': images[i],
                    'next': images[i+1],
                    'seq_path': seq_folder
                })
        return triplets

    def __getitem__(self, idx):
        t = self.samples[idx]
        
        # Load images
        img_curr = Image.open(t['curr']).convert('RGB')
        img_prev = Image.open(t['prev']).convert('RGB')
        img_next = Image.open(t['next']).convert('RGB')

        # Get relative 4D timestamps
        rel_t = self.projector.get_relative_timestamp(t['curr'].name)
        
        return {
            'images': torch.stack([img_prev, img_curr, img_next]), # (3, C, H, W)
            'rel_t': rel_t,
            'seq_name': t['seq_path'].name
        }

### *4. Initialize setup*

In [7]:
import json
import os
import yaml
import torch
import re
from pathlib import Path

# --- Correct Pathing ---
base_path = Path(r"C:\Users\maxlars\UGV_research_pipeline\Goose_dataset")

# The schema is in the 'common' folder
spec_schema_path = base_path / "SpecMetaData.json" 

# Calibration is in its own dedicated folder
calibration_file = base_path / "calibration" / "windshield_vis.yaml" 

# Your processed chronological images
image_root = base_path / "processed_rgb" 

# --- 1. Load the JSON Schema ---
with open(spec_schema_path, 'r') as f:
    spec_schema = json.load(f)
    print(f"Loaded schema: {spec_schema['title']}") #

# --- 2. Iterative Sequence Processing ---
for sequence_folder in sorted(image_root.iterdir()):
    if not sequence_folder.is_dir():
        continue
        
    # Metadata file for the specific sequence
    sequence_metadata_path = sequence_folder / "metadata.yaml"
    
    if not sequence_metadata_path.exists():
        continue

    with open(sequence_metadata_path, 'r') as f:
        seq_metadata = yaml.safe_load(f)
    
    # Extract the 'time_machine' UNIX epoch (ns) iteratively
    start_ns = seq_metadata['info']['time_machine'] 
    
    # Initialize Projector with the sequence-specific temporal anchor
    projector = GooseQueryProjector(
        yaml_path=calibration_file, 
        sequence_start_time_ns=start_ns
    )
    
    print(f"\nProcessing Sequence: {sequence_folder.name}")
    
    # Get sorted files for chronological 4D learning
    image_files = sorted([f for f in os.listdir(sequence_folder) if "rgb" in f])
    
    for fname in image_files[:2]:
        rel_t = projector.get_relative_timestamp(fname)
        print(f"  Image: {fname} | 4D T-Feature: {rel_t:.4f}s")

Loaded schema: SpecMetaData.json


### *5. Model Initialization*

In [ ]:
class SelfSupervisedOccupancyLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1_loss = nn.L1Loss()

    def forward(self, pred_image, target_image):
        """
        pred_image: The image reconstructed by projecting 3D queries into time T+1
        target_image: The actual RGB image at time T+1
        """
        # 1. Photometric Loss (L1)
        l1 = self.l1_loss(pred_image, target_image)
        
        # 2. Add SSIM for structural consistency (optional but recommended)
        # ssim_loss = 1 - ssim(pred_image, target_image)
        
        return l1